Installing the necessary pip packages

In [1]:
! pip install lxml

Importing libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import lxml.html as lh
import requests
print('Libraries imported.')

Libraries imported.


Obtaining data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
status_code = page.status_code # 200
content = page.content # .html file

Extracting elements from the html file

In [4]:
doc = lh.fromstring(content)
tr_elements = doc.xpath('//tr') # to get the rows

Extracting the first row element as the column names of the dataframe

In [5]:
cols = []
for (index,tr) in enumerate(tr_elements[0]):
    cols.append((tr.text_content(),[]))

Adding data based on the column 

In [6]:
for row_index in range(1,len(tr_elements)):
    row = tr_elements[row_index]
    
    if len(row) != 3:
        break
    col_index = 0
    for row_data in row.iterchildren():
        data = row_data.text_content()
        # adding data to each column
        cols[col_index][1].append(data)
        col_index +=1       

Creating the dataframe and filling in data

In [7]:
dict = {key:value for (key,value) in cols}
dict.keys() #dict_keys(['Postal Code\n', 'Borough\n', 'Neighbourhood\n'])
df=pd.DataFrame(dict)
df.columns = ['Postal Code', 'Borough', 'Neighbourhood']
df = df.replace(r'\n','',regex=True) # removing the '\n'
df.drop(df.tail(1).index,inplace=True) # removing the last row
df = df[df.Borough != "Not assigned"] # removing the 'Not assigned'
df = df.set_index('Postal Code')

If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.

In [8]:
for index,row in df.iterrows():
#     print(row["Neighbourhood"])
#     print(row["Borough"])
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row ["Borough"]

Saving the data to .csv file

In [9]:
df.to_csv('Toronto.csv')

In [10]:
toronto = pd.read_csv('Toronto.csv')

In [11]:
toronto

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Printing the number of rows of the dataframe

In [12]:
print(toronto.shape)

(103, 3)
